## Outline
* [leetcode 202: Happy Number](#202)

<a id="202"></a>
### 202. Happy Number
* 要注意 cycle! 例如：    

<img src=img/array_1.png width=500x>
* 會有 3 種情況：

    1. 等於 1
    2. 形成 cycle
    3. 越來越大，變成無限大  
  但其實第 3 種狀況不會發生，因為如下表所示，  
 <img src=img/array_2.png width=500x>   
 在題目的數量級底下，所有的數字 digits 運算後都會回到 3 位數，故只可能是形成小於 243 的 cycle，或者回到 1。所以我們不需要處理狀況 3。 
 因此，我們只要設計兩個部分：
 1. 目前這個數經過運算後，會變成什麼數字？
 2. 偵測是否進入 cycle 

In [3]:
# Approach 1: 簡單版
def isHappy(self, n: int) -> bool:

    # 第一部分：使用 divmod 得到運算後的結果
    def get_next(n):
        total_sum = 0
        while n > 0:
            n, digit = divmod(n, 10)
            total_sum += digit ** 2
        return total_sum

    # 第二部分：使用 set 紀錄看過的數字，若再次出現即進入 cycle
    seen = set()
    while n != 1 and n not in seen:
        seen.add(n)
        n = get_next(n)

    return n == 1

* 時間複雜度：   
1. 第一部分：處理每個 digit 是 O(1)，共有 logn 個 digit，故 O(longn)
2. 根據上面的討論，當運算結果 < 243，那麼就不可能再高於 243 了，並且最多只有 3 個 digits。
3. 對於某個 n > 243，考慮運算過程的整個 chain，worse case 是 O(logn) + O(loglogn) + O(logloglogn)+....， dominent 項是 O(logn)，所以整體的時間複雜度為 O(logn)。
* 空間複雜度：
1. 需要用一個 set 來存，大小決定於會有多少 n 被存進 set 裡面、 n 有多大。會跟著 time complexity 成長，故也是 O(logn)。

### Approach 2: Floyd's Cycle-Finding Algorithm
想像龜兔賽跑，兔子跑比較快。但是不管龜兔的起始點在哪裡，在 cycle 中他們終究會相遇，因為兔子跑比較快，會逐漸縮短和烏龜的差距。  

所以我們可以用兩個 tracker，一個代表烏龜，一個代表兔子，在每一個 round 中，烏龜跑一步，兔子跑兩步（call 兩次 get_next）。  

假如沒有 cycle 存在，那兔子會先到達 n = 1的結果。假如有 cycle 存在，那龜兔就會相遇（在同一個數字上）。

In [5]:
def isHappy(self, n: int) -> bool:

    # 第一部分：使用 divmod 得到運算後的結果
    def get_next(n):
        total_sum = 0
        while n > 0:
            n, digit = divmod(n, 10)
            total_sum += digit ** 2
        return total_sum

    # 第二部分：使用 Floyd's Cycle-Finding Algorithm
    tortoise = n
    hare = self.get_next(n)
    while tortoise != hare and hare != 1:
        tortoise = self.get_next(tortoise)
        hare = self.get_next(self.get_next(hare))

    return hare == 1

### [15. 3 sum](https://leetcode.com/problems/3sum/)
* Reference: https://leetcode.com/problems/3sum/discuss/7498/Python-solution-with-detailed-explanation  
* 目標是把 3sum reduce 成 2sum。 2sum: x + y = 0 。 則在 3sum 的問題中， 把 z 固定，則 x + y = -z 就可以 reduce 成 2sum。
* 另外一個要處理的問題是去除重複的解。
    * 在 2sum 中去除重複：
        1. 先 sort array
        2. 假設找到一組解是 nums[s] + nums[e] = 0，則假如 nums[s+1] = nums[s], 那 nums[s+1] 也會找到 nums[e]，形成一組重複解。所以應該跳過重複的數。例如： nums = [-2, -1, -1, 0, 1, 2]，當找到  [-2, `-1`, -1, 0, `1`, 2] ，要再往下找時，就應該跳過第二個 -1，直接從 0 開始找，否則第二個 -1 也會找到 [-1, 1] 這組重複解。
    * 在 3sum 中去除重複：
        1. 一樣 sort array
        2. 和 2sum 同樣道理，假如要去除重複，則在 iterate z 的時候，當遇到相同的 z 就應該直接跳過。
        3. 方法二：假如 2sum 是用 dict 存走過的路，在這裡一樣可以用。差別是要去除重複。做法是假如已經存過一組解了，就把這個 key 的 value 設為 None。假如下一次遇到 key 是 None 的就知道他用過了，就跳過他。  
        4. 方法三：在做 2sum 時，直接確認目前的數字總和是多少，假如太大，代表右邊 index 要往左移，太小則是左邊 index 要往右移，假如剛好等於 0，代表是一組解，儲存起來之後，左右 index 各往內一格。用 set() 來儲存答案，就不需要 check 重複！
        
[Python Notes] `dict()` 和 `set()` 都是 hash table, 取值都是 O(1)，差別在於 set() 不需要有 value。
            

In [2]:
### 法一
class Solution:
    def threeSum(self, nums):

        """
        :type nums: List[int]
        :rtype: List[List[int]]
        """
        nums.sort()
        N, result = len(nums), []
        for i in range(N):
            if i > 0 and nums[i] == nums[i-1]:
                continue
            target = nums[i]*-1
            s,e = i+1, N-1
            while s<e:
                if nums[s]+nums[e] == target:
                    result.append([nums[i], nums[s], nums[e]])
                    s = s+1
                    while s<e and nums[s] == nums[s-1]:
                        s = s+1
                elif nums[s] + nums[e] < target:
                    s = s+1
                else:
                    e = e-1
        return result


In [3]:
### 法二

class Solution:
    def threeSum(self, nums):
        if len(nums) < 3:
            return []
        

        nums.sort()
        result = []
        
        for z in range(len(nums)):
            target = -1 * nums[z]
            
            if z > 0 and nums[z] == nums[z-1]:
                continue
            
            temp_dict = {}
            for x in range(z+1, len(nums)):
                
                if nums[x] in temp_dict and temp_dict[nums[x]] != None:
                    result.append([nums[z], temp_dict[nums[x]], nums[x]])
                    temp_dict[nums[x]] = None
                    
                elif nums[x] in temp_dict and temp_dict[nums[x]] == None:
                    continue
                
                else:
                    temp_dict[target-nums[x]] = nums[x]
        
        
        return result

In [4]:
### 法三
class Solution:
    def threeSum(self):

        """
        :type nums: List[int]
        :rtype: List[List[int]]
        """
        nums.sort()
        tri = set()
        ans = []
        for i in range(len(nums)-2):
            if nums[i] > 0:
                break
            
            if i > 0 and nums[i] == nums[i-1]:
                continue
            
            l, r = i + 1, len(nums) - 1
            
            while l < r:
                total = nums[i] + nums[l] + nums[r]
            
                if total > 0:
                    r -= 1
                elif total < 0:
                    l += 1
            
                else:
                    tri.add((nums[i], nums[l], nums[r]))
                    l += 1
                    r -= 1
                
        
        return [list(t) for t in tri]